### Análise dos dados do dataset oficial para vizualização dos cruzamentos ocorridos.

In [270]:
# Importing libraries
import pandas as pd
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt
import math

In [271]:
df = pd.read_csv('../dados/dados_brutos.csv') # Leitura dos dados
df = df.sort_values(by=['FRAME', 'TIME_JOGADOR']) # Ordena os dados por frame e tempo do jogador

In [272]:
df.columns

Index(['TEMPO_JOGO', 'FRAME', 'ID_JOGADOR', 'NOME_JOGADOR', 'POSICAO_JOGADOR',
       'TIME_JOGADOR', 'OBJETO_OBSERVADO', 'X', 'Y', 'Z', 'VELOCIDADE',
       'TIME_POSSE'],
      dtype='object')

In [273]:
df_bola = df[df['OBJETO_OBSERVADO'] == 'bola']

# Cria colunas para indicar se a bola está dentro da área
df_bola["DENTRO_AREA_DIREITA"] = 0
df_bola["DENTRO_AREA_ESQUERDA"] = 0

area_direita = (df_bola["X"] > 101.5) & (df_bola["Y"] > 17.85) & (df_bola["Y"] < 58.15)
df_bola.loc[area_direita, "DENTRO_AREA_DIREITA"] = 1

area_esquerda = (df_bola["X"] < 16.5) & (df_bola["Y"] > 17.85) & (df_bola["Y"] < 58.15)
df_bola.loc[area_esquerda, "DENTRO_AREA_ESQUERDA"] = 1

C:\Users\Daniel\AppData\Local\Temp\ipykernel_17304\2553525433.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bola["DENTRO_AREA_DIREITA"] = 0
C:\Users\Daniel\AppData\Local\Temp\ipykernel_17304\2553525433.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bola["DENTRO_AREA_ESQUERDA"] = 0


In [274]:
# DataFrame com os dados da bola
df_bola = df_bola.sort_values(by=['TEMPO_JOGO', 'FRAME'])
df_bola.reset_index(drop=True, inplace=True)

In [275]:
# Palmeiras atacando a área direita

df_pal_ataque_direita = df_bola[
    (df_bola['TIME_POSSE'] == 'PAL') & 
    (df_bola['VELOCIDADE'] >= 1.5) & 
    (df_bola['TEMPO_JOGO'] == 1) & 
    (df_bola['X'] < 118) & 
    (df_bola['X'] >= 86.5) & 
    (df_bola['Y'] < 76)
]

df_pal_ataque_direita.reset_index(drop=True, inplace=True)

areas_direita = {
    'E1.2':{
        'X0': 86.5,
        'X1': 101.5,
        'Y0': 0,
        'Y1': 17.85,
        'num': 0
    },
    'E1.1':{
        'X0': 86.5,
        'X1': 101.5,
        'Y0': 17.85,
        'Y1': 38,
        'num': 0
    },
    'E2.1':{
        'X0': 101.5,
        'X1': 109.75,
        'Y0': 0,
        'Y1': 17.85,
        'num': 0
    },
    'E2.2':{
        'X0': 109.75,
        'X1': 118,
        'Y0': 0,
        'Y1': 17.85,
        'num': 0
    },
    'E3':{
        'X0': 101.5,
        'X1': 109.75,
        'Y0': 17.85,
        'Y1': 27.92,
        'num': 0
    },
    'D1.1':{
        'X0': 86.5,
        'X1': 101.5,
        'Y0': 38,
        'Y1': 58.15,
        'num': 0
    },
    'D1.2':{
        'X0': 86.5,
        'X1': 101.5,
        'Y0': 58.15,
        'Y1': 76,
        'num': 0
    },
    'D2.1':{
        'X0': 101.5,
        'X1': 109.75,
        'Y0': 58.15,
        'Y1': 76,
        'num': 0
    },
    'D2.2':{
        'X0': 109.75,
        'X1': 118,
        'Y0': 58.15,
        'Y1': 76,
        'num': 0
    },
    'D3':{
        'X0': 101.5,
        'X1': 109.75,
        'Y0': 48.07,
        'Y1': 58.15,
        'num':0
    }
}

In [276]:
# Variáveis para o simulação inicial de cruzamentos
contador = 0
frames_teste = []


In [277]:
# Análise inicial de cruzamentos

for index, row in df_pal_ataque_direita.iterrows():
    if index+1 == len(df_pal_ataque_direita): # Verifica se é o último frame
        break
    if df_pal_ataque_direita.iloc[index+1]['DENTRO_AREA_DIREITA'] == 1 and row['DENTRO_AREA_DIREITA'] == 0: # Verifica se a bola entrou na área
        sequencia_de_1 = 0
        for i in range(1, 21): # 0.8 segundo
            if index+i == len(df_pal_ataque_direita):
                break
            if df_pal_ataque_direita.iloc[index+i]['DENTRO_AREA_DIREITA'] == 1:
                sequencia_de_1 += 1
            if sequencia_de_1 == 20: # A sequência de 20 frames significa que a bola esteve na área por 0.8 segundo
                for area in areas_direita:
                    if df_pal_ataque_direita.iloc[index-50]['X'] > areas_direita[area]['X0'] and df_pal_ataque_direita.iloc[index-50]['X'] < areas_direita[area]['X1'] and df_pal_ataque_direita.iloc[index-50]['Y'] > areas_direita[area]['Y0'] and df_pal_ataque_direita.iloc[index-50]['Y'] < areas_direita[area]['Y1']:
                        areas_direita[area]['num'] += 1
                        contador += 1
                        frames_teste.append(df_pal_ataque_direita.iloc[index+1]['FRAME']) # frame quando a bola entra na área
                        break
                break

# Resultados
print(f'Primeira simulação de cruzamentos resultou em: {contador}')
print(f'Primeira simulação de frames resultou em: {frames_teste}')

Primeira simulação de cruzamentos resultou em: 19
Primeira simulação de frames resultou em: [8189, 14138, 15308, 19367, 19484, 20363, 22567, 27621, 29742, 39832, 40645, 48331, 49319, 49455, 50587, 52091, 52427, 62235, 67044]


In [278]:
# DataFrames com somente as informações da bola e dos jogadores de cada time
df_palmeiras = df[(df['TIME_JOGADOR'] != 'RED')]
df_bragantino = df[(df['TIME_JOGADOR'] != 'PAL')]

In [279]:
# DataFrames com somente as informações da bola e dos jogadores de cada time no primeiro tempo
df_palmeiras_t1 = df_palmeiras[df_palmeiras['TEMPO_JOGO'] == 1]
df_bragantino_t1 = df_bragantino[df_bragantino['TEMPO_JOGO'] == 1]


In [280]:
# Dicionários
jogadores_que_cruzaram = {} # Dicionário com os jogadores que cruzaram a bola / frames que cruzaram a bola
velocidades_maximas = {} # Dicionário com as velocidades máximas da bola em cada frame que ocorre um cruzamento

# Variáveis
cruzamentos = 0 # Número de cruzamentos final
frames_finais = [] # Lista com os frames finais de cada cruzamento

In [281]:
# Análise de quem realizou cada cruzamento contado em 'frames'

for frame in frames_teste: 

    # Variáveis
    x_bola = 0
    y_bola = 0
    velocidade_bola = 0
    distancia_entre_jogador_e_bola = 100 # Valor inicial alto para que qualquer distância seja menor que ele
    
    # DataFrame com os dados da bola no intervalo do cruzamento (frame e 2 segundos antes)
    df_frame_bola = df_palmeiras_t1[(df_palmeiras_t1['FRAME'] <= frame) & (df_palmeiras_t1['FRAME'] >= frame-50) & (df_palmeiras_t1['OBJETO_OBSERVADO'] == 'bola')]

    for index, row in df_frame_bola.iterrows():
        if row['VELOCIDADE'] > velocidade_bola:
            velocidade_bola = row['VELOCIDADE'] # Determina a velocidade máxima da bola no intervalo
            frame_que_cruzou = row['FRAME'] # Determina o frame em que a bola atinge a velocidade máxima
            # Coordenadas da bola no momento do cruzamento
            x_bola = row['X']
            y_bola = row['Y']

    # Velocidades máximas atingidas pela bola em cada cruzamento (momento em que o jogador toca na bola)
    velocidades_maximas[frame_que_cruzou] = velocidade_bola

    # Mesmo DataFrame, mas com os dados dos jogadores
    df_frame_jogadores = df_palmeiras_t1[(df_palmeiras_t1['FRAME'] <= frame) & (df_palmeiras_t1['FRAME'] >= frame-50) & (df_palmeiras_t1['OBJETO_OBSERVADO'] == 'jogador')]

    for index, row in df_frame_jogadores.iterrows():
        pos_jogador = [row['X'], row['Y']] # Coordenadas do jogador
        pos_bola = [x_bola, y_bola] # Coordenadas da bola
        distancia_medida = math.dist(pos_jogador, pos_bola) # Distância entre o jogador e a bola: distância de dois pontos
        if distancia_medida < distancia_entre_jogador_e_bola: # A menor distância é a que o jogador toca na bola (cruza)
            distancia_entre_jogador_e_bola = distancia_medida
            jogador = row['NOME_JOGADOR'] # Nome do jogador que cruzou
    
    for area in areas_direita:
        # Verifica se o cruzamento ocorreu em alguma das zonas
        if x_bola > areas_direita[area]['X0'] and x_bola < areas_direita[area]['X1'] and y_bola > areas_direita[area]['Y0'] and y_bola < areas_direita[area]['Y1']:
            areas_direita[area]['num'] += 1
            cruzamentos += 1
            frames_finais.append(frame_que_cruzou)
            # Adiciona o jogador que cruzou a bola no dicionário
            if jogador in jogadores_que_cruzaram:
                jogadores_que_cruzaram[jogador].append(frame_que_cruzou)
            else:
                jogadores_que_cruzaram[jogador] = [frame_que_cruzou]
            break

# Resultados finais
print(f'Número de cruzamentos encontrados no 1º tempo do Palmeiras foi: {cruzamentos}')
print(f'Nos frames: {frames_finais}')
print(f'Dicionário dos jogadores que cruzaram a bola, com os frames: {jogadores_que_cruzaram}')
print(cruzamentos)


Número de cruzamentos encontrados no 1º tempo do Palmeiras foi: 15
Nos frames: [8183, 15268, 19341, 19484, 20358, 22566, 27600, 29742, 39832, 48329, 49309, 52088, 52423, 62186, 67040]
Dicionário dos jogadores que cruzaram a bola, com os frames: {'Raphael Veiga': [8183, 19484, 20358, 22566, 49309], 'Rony': [15268], 'Artur': [19341, 27600, 52423], 'Vanderlan': [29742, 62186, 67040], 'Dudu': [39832, 48329, 52088]}
15


In [282]:
participacao_por_cruzamento_pal_nome = {}

participacao_por_cruzamento_pal_quantidade = {}




frames_cruzamento = [8183,14134,15268,19341,19484,20358,22566,27600,29742,39832,40628,48329,49309,49415,50545,52088,52423,62186,67040]
for frame in frames_cruzamento:

    df_bola = df_palmeiras_t1[(df_palmeiras_t1['OBJETO_OBSERVADO'] == 'bola') & (df_palmeiras_t1['FRAME'] == frame)]
    x = df_bola['X'].iloc[0]
    df_frame_jogadores = df_palmeiras_t1[(df_palmeiras_t1['FRAME'] >= frame ) & (df_palmeiras_t1['FRAME'] <= frame + 50) & (df_palmeiras_t1['OBJETO_OBSERVADO'] == 'jogador')]
    area_direita = (df_frame_jogadores["X"] > 101.5) & (df_frame_jogadores["Y"] > 17.85) & (df_frame_jogadores["Y"] < 58.15)

    df_jogadores_area_direita = df_frame_jogadores[area_direita]

    nomes_jogadores_area_direita = df_jogadores_area_direita['NOME_JOGADOR'].unique().tolist()
    participacao_por_cruzamento_pal_nome[frame] = nomes_jogadores_area_direita
    participacao_por_cruzamento_pal_quantidade[frame] = len(nomes_jogadores_area_direita)



print(participacao_por_cruzamento_pal_nome)


print(participacao_por_cruzamento_pal_quantidade)



{8183: ['Rony', 'Artur'], 14134: [], 15268: ['Dudu', 'Gabriel Menino'], 19341: ['Rony', 'Raphael Veiga', 'Dudu'], 19484: ['Artur', 'Rony', 'Raphael Veiga', 'Dudu'], 20358: ['Luan', 'Rony', 'Gustavo Gomez'], 22566: ['Dudu', 'Rony', 'Gustavo Gomez', 'Vanderlan'], 27600: ['Rony', 'Dudu', 'Mayke'], 29742: ['Rony', 'Vanderlan', 'Raphael Veiga'], 39832: ['Raphael Veiga', 'Rony'], 40628: ['Artur', 'Rony', 'Luan'], 48329: ['Vanderlan', 'Artur', 'Raphael Veiga', 'Rony'], 49309: ['Gustavo Gomez', 'Artur', 'Rony', 'Fabinho'], 49415: ['Gustavo Gomez', 'Fabinho', 'Rony', 'Artur'], 50545: ['Artur', 'Rony', 'Fabinho', 'Raphael Veiga'], 52088: ['Artur', 'Rony', 'Gustavo Gomez'], 52423: ['Rony', 'Luan', 'Gustavo Gomez'], 62186: ['Rony', 'Vanderlan'], 67040: ['Rony']}
{8183: 2, 14134: 0, 15268: 2, 19341: 3, 19484: 4, 20358: 3, 22566: 4, 27600: 3, 29742: 3, 39832: 2, 40628: 3, 48329: 4, 49309: 4, 49415: 4, 50545: 4, 52088: 3, 52423: 3, 62186: 2, 67040: 1}


In [283]:
participacao_por_cruzamento_red_nome = {}

participacao_por_cruzamento_red_quantidade = {}


frames_cruzamento = [8183,14134,15268,19341,19484,20358,22566,27600,29742,39832,40628,48329,49309,49415,50545,52088,52423,62186,67040]
for frame in frames_cruzamento:

    df_bola = df_bragantino_t1[(df_bragantino_t1['OBJETO_OBSERVADO'] == 'bola') & (df_bragantino_t1['FRAME'] == frame)]
    x = df_bola['X'].iloc[0]
    df_frame_jogadores_bragantino = df_bragantino_t1[(df_bragantino_t1['FRAME'] >= frame ) & (df_bragantino_t1['FRAME'] <= frame + 50) & (df_bragantino_t1['OBJETO_OBSERVADO'] == 'jogador')]
    area_direita = (df_frame_jogadores_bragantino["X"] > 101.5) & (df_frame_jogadores_bragantino["Y"] > 17.85) & (df_frame_jogadores_bragantino["Y"] < 58.15) & (df_frame_jogadores_bragantino["POSICAO_JOGADOR"] != "Goleiro")

    df_jogadores_area_direita_bragantino = df_frame_jogadores_bragantino[area_direita]

    nomes_jogadores_area_direita_bragantino = df_jogadores_area_direita_bragantino['NOME_JOGADOR'].unique().tolist()
    participacao_por_cruzamento_red_nome[frame] = nomes_jogadores_area_direita_bragantino
    participacao_por_cruzamento_red_quantidade[frame] = len(nomes_jogadores_area_direita_bragantino)

print(participacao_por_cruzamento_red_nome)
print(participacao_por_cruzamento_red_quantidade)

{8183: ['Eduardo Santos', 'Natan', 'Jose Hurtado', 'Helinho'], 14134: [], 15268: ['Eduardo Santos', 'Jose Hurtado', 'Ramires', 'Matheus Fernandes'], 19341: ['Natan', 'Eduardo Santos', 'Jose Hurtado', 'Matheus Fernandes'], 19484: ['Ramires', 'Matheus Fernandes', 'Natan', 'Jose Hurtado', 'Eduardo Santos', 'Juninho Capixaba', 'Lucas Evangelista', 'Bruno'], 20358: ['Jose Hurtado', 'Eduardo Santos', 'Natan', 'Ramires'], 22566: ['Natan', 'Matheus Fernandes', 'Jose Hurtado', 'Juninho Capixaba', 'Eduardo Santos', 'Lucas Evangelista', 'Helinho', 'Eduardo Sasha', 'Ramires', 'Bruno'], 27600: ['Natan', 'Eduardo Santos', 'Jose Hurtado'], 29742: ['Juninho Capixaba', 'Natan', 'Jose Hurtado', 'Eduardo Santos', 'Matheus Fernandes', 'Lucas Evangelista'], 39832: ['Matheus Fernandes', 'Juninho Capixaba', 'Jose Hurtado', 'Natan', 'Eduardo Santos'], 40628: ['Matheus Fernandes', 'Natan', 'Bruno', 'Juninho Capixaba', 'Eduardo Sasha', 'Helinho', 'Jose Hurtado', 'Eduardo Santos'], 48329: ['Matheus Fernandes', '

In [284]:
correspondencia_nome_num_pal = {
'Weverton': 21,
'Mayke': 12,
'Gustavo Gomez': 15,
'Luan': 13,
'Vanderlan': 6,
'Fabinho': 35,
'Richard Ríos': 27,
'Gabriel Menino': 25,
'Raphael Veiga': 23,
'Flaco López': 18,
'Artur': 14,
'Bruno Tabata': 11,
'Rony': 10,
'Endrick': 9,
'Dudu': 7,
'Breno Lopez': 19,
'Marcelo Lomba': 42,
'Gustavo Garcia': 32,
'Naves': 34,
'Jailson': 30,
'Luis Guilherme': 31,
'Jhon Jhon': 40,
'Rafael Navarro': 29
}

correspondencia_nome_num_red = {
'Lucão': 40,
'Luan Patrick': 30,
'Guilherme Lopes': 31,
'Gustavinho': 22,
'Praxedes': 25,
'Sorriso': 27,
'Everton': 42,
'Cleiton Schwengber': 1,
'Jose Hurtado': 34,
'Aderlan': 13,
'Eduardo Santos': 26,
'Natan': 4,
'Realpe': 2,
'Juninho Capixaba': 29,
'Matheus Fernandes': 35,
'Ramires': 7,
'Lucas Evangelista': 8,
'Alerrandro': 9,
'Bruno': 17,
'Vitinho': 28,
'Eduardo Sasha': 19,
'Helinho': 11,
'Henry Mosquera': 16
}



In [285]:
participacao_por_cruzamento_numeros_pal = {}

for frame, jogadores in participacao_por_cruzamento_pal_nome.items():
    
    numeros_jogadores = [correspondencia_nome_num_pal.get(nome, nome) for nome in jogadores]

    participacao_por_cruzamento_numeros_pal[frame] = numeros_jogadores

print(participacao_por_cruzamento_numeros_pal)

{8183: [10, 14], 14134: [], 15268: [7, 25], 19341: [10, 23, 7], 19484: [14, 10, 23, 7], 20358: [13, 10, 15], 22566: [7, 10, 15, 6], 27600: [10, 7, 12], 29742: [10, 6, 23], 39832: [23, 10], 40628: [14, 10, 13], 48329: [6, 14, 23, 10], 49309: [15, 14, 10, 35], 49415: [15, 35, 10, 14], 50545: [14, 10, 35, 23], 52088: [14, 10, 15], 52423: [10, 13, 15], 62186: [10, 6], 67040: [10]}


In [286]:
participacao_por_cruzamento_numeros_red = {}

for frame, jogadores in participacao_por_cruzamento_red_nome.items():
    
    numeros_jogadores = [correspondencia_nome_num_red.get(nome, nome) for nome in jogadores]

    participacao_por_cruzamento_numeros_red[frame] = numeros_jogadores

print(participacao_por_cruzamento_numeros_red)

{8183: [26, 4, 34, 11], 14134: [], 15268: [26, 34, 7, 35], 19341: [4, 26, 34, 35], 19484: [7, 35, 4, 34, 26, 29, 8, 17], 20358: [34, 26, 4, 7], 22566: [4, 35, 34, 29, 26, 8, 11, 19, 7, 17], 27600: [4, 26, 34], 29742: [29, 4, 34, 26, 35, 8], 39832: [35, 29, 34, 4, 26], 40628: [35, 4, 17, 29, 19, 11, 34, 26], 48329: [35, 26, 4, 8, 34, 17, 11], 49309: [35, 17, 29, 11, 34, 4, 7, 26, 8], 49415: [35, 26, 34, 7, 4], 50545: [4, 26, 34, 11], 52088: [34, 29, 8, 4, 19, 26, 35, 7], 52423: [35, 8, 26, 4, 11, 29], 62186: [26, 4, 34, 29], 67040: [4, 26, 34, 29]}


In [294]:
def contar_aparicoes_jogadores(dicionario):
    contagem_jogadores = {}

    for jogadores in dicionario.values():
        for jogador in jogadores:
            contagem_jogadores[jogador] = contagem_jogadores.get(jogador, 0) + 1

    return contagem_jogadores

print(contar_aparicoes_jogadores(participacao_por_cruzamento_pal_nome))

print(contar_aparicoes_jogadores(participacao_por_cruzamento_red_nome))



{'Rony': 17, 'Artur': 8, 'Dudu': 5, 'Gabriel Menino': 1, 'Raphael Veiga': 6, 'Luan': 3, 'Gustavo Gomez': 6, 'Vanderlan': 4, 'Mayke': 1, 'Fabinho': 3}
{'Eduardo Santos': 18, 'Natan': 17, 'Jose Hurtado': 17, 'Helinho': 7, 'Ramires': 7, 'Matheus Fernandes': 12, 'Juninho Capixaba': 10, 'Lucas Evangelista': 7, 'Bruno': 5, 'Eduardo Sasha': 3}


In [ ]:
# Desfecho dos cruzamentos (Bem-Sucessido, Bloqueado, Perdido)

velocidade_pico = 0
frames_e_frames_velocidade_pico = {}

# Analisar os frames para encontrar velocidade pico
for frame in frames_finais:
    df_frame_bola = df_palmeiras_t1[(df_palmeiras_t1['FRAME'] >= frame) & (df_palmeiras_t1['FRAME'] <= frame + 100) & (df_palmeiras_t1['OBJETO_OBSERVADO'] == 'bola')]
    df_frame_bola = df_frame_bola.reset_index(drop=True)

    for index, row in df_frame_bola.iterrows():
        if index + 1 == len(df_frame_bola):
            break  # Verifica se é o último frame

        velocidade_atual_registrada = row['VELOCIDADE']
        if df_frame_bola.iloc[index + 1]['VELOCIDADE'] - velocidade_atual_registrada > 2:
            velocidade_pico = df_frame_bola.iloc[index + 1]['VELOCIDADE']
            frames_e_frames_velocidade_pico[frame] = df_frame_bola.iloc[index + 1]['FRAME']
            break

print("Frames com velocidade pico:", frames_e_frames_velocidade_pico)

df_t1 = df[df['TEMPO_JOGO'] == 1]

# Verifica se o cruzamento foi bem-sucedido, bloqueado ou perdido
desfecho_cruzamentos = {}

for frame, frame_velocidade_pico in frames_e_frames_velocidade_pico.items():
    df_frame_bola = df_t1[(df_t1['FRAME'] == frame_velocidade_pico) & (df_t1['OBJETO_OBSERVADO'] == 'bola')]
    df_frame_jogadores = df_t1[(df_t1['FRAME'] == frame_velocidade_pico) & (df_t1['OBJETO_OBSERVADO'] == 'jogador')]

    if not df_frame_bola.empty and not df_frame_jogadores.empty:
        pos_bola = [df_frame_bola.iloc[0]['X'], df_frame_bola.iloc[0]['Y']]

        for index, row in df_frame_jogadores.iterrows():
            pos_jogador = [row['X'], row['Y']]
            distancia_medida = math.dist(pos_jogador, pos_bola)

            if distancia_medida < 1.5:
                if row['TIME_JOGADOR'] == 'PAL':
                    desfecho_cruzamentos[frame] = 'Bem-Sucedido'
                elif row['TIME_JOGADOR'] == 'RED':
                    desfecho_cruzamentos[frame] = 'Bloqueado'
                break
        else:  # Ninguém encostou na bola
            desfecho_cruzamentos[frame] = 'Perdido'
    else:
        desfecho_cruzamentos[frame] = 'Perdido'  # Se não há dados disponíveis para o frame, consideramos como perdido

for frame in frames_finais:
    if frame not in desfecho_cruzamentos:
        desfecho_cruzamentos[frame] = 'Perdido'

desfecho_cruzamentos = dict(sorted(desfecho_cruzamentos.items())) # Ordena o dicionário por ordem crescente de frames

print("Desfecho final dos cruzamentos:", desfecho_cruzamentos)